# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and implementation of two (simple) techniques that are enough to solve sudoku in a reasonable time.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [1]:
#notation

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

DIGITS   = '123456789'
ROWS     = 'ABCDEFGHI'
COLS     = DIGITS
SQUARES  = cross(ROWS, COLS)
UNITLIST = ([cross(ROWS, c) for c in COLS] +
            [cross(r, COLS) for r in ROWS] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
UNITS = dict((s, [u for u in UNITLIST if s in u]) 
             for s in SQUARES)
PEERS = dict((s, set(sum(UNITS[s],[]))-set([s]))
             for s in SQUARES)

global ASSIGNMENTS
ASSIGNMENTS = []

print(UNITS['A1'])

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]


### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:



In [2]:
# parser
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in DIGITS or c in '0.']
    assert len(chars) == 81
    return dict(zip(SQUARES, chars))


def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in SQUARES)
    line = '+'.join(['-'*(width*3)]*3)
    for r in ROWS:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in COLS))
        if r in 'CF': print(line)
    print()

# sample sudoku
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
display(grid_values(grid2))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 



### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [3]:
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, DIGITS) for s in SQUARES)
    for s,d in grid_values(grid).items():
        if d in DIGITS and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (grid_values(grid2))
                 
print(sudoku_values)

{'A1': '4', 'A2': '.', 'A3': '.', 'A4': '.', 'A5': '.', 'A6': '.', 'A7': '8', 'A8': '.', 'A9': '5', 'B1': '.', 'B2': '3', 'B3': '.', 'B4': '.', 'B5': '.', 'B6': '.', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '.', 'C3': '.', 'C4': '7', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '.', 'C9': '.', 'D1': '.', 'D2': '2', 'D3': '.', 'D4': '.', 'D5': '.', 'D6': '.', 'D7': '.', 'D8': '6', 'D9': '.', 'E1': '.', 'E2': '.', 'E3': '.', 'E4': '.', 'E5': '8', 'E6': '.', 'E7': '4', 'E8': '.', 'E9': '.', 'F1': '.', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '1', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '.', 'G1': '.', 'G2': '.', 'G3': '.', 'G4': '6', 'G5': '.', 'G6': '3', 'G7': '.', 'G8': '7', 'G9': '.', 'H1': '5', 'H2': '.', 'H3': '.', 'H4': '2', 'H5': '.', 'H6': '.', 'H7': '.', 'H8': '.', 'H9': '.', 'I1': '1', 'I2': '.', 'I3': '4', 'I4': '.', 'I5': '.', 'I6': '.', 'I7': '.', 'I8': '.', 'I9': '.'}


# Constraint Propagation - original code
The function parse_grid calls assign(values, s, d). We could implement this as values[s] = d, but we can do more than just that. Those with experience solving Sudoku puzzles know that there are two important strategies that we can use to make progress towards filling in all the squares:

(1) If a square has only one possible value, then eliminate that value from the square's peers. 

(2) If a unit has only one possible place for a value, then put the value there.

As an example of strategy (1) if we assign 7 to A1, yielding {'A1': '7', 'A2':'123456789', ...}, we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers), giving us {'A1': '7', 'A2': '12345689', ...}. As an example of strategy (2), if it turns out that none of A3 through A9 has a 3 as a possible value, then the 3 must belong in A2, and we can update to {'A1': '7', 'A2':'3', ...}. These updates to A2 may in turn cause further updates to its peers, and the peers of those peers, and so on. This process is called constraint propagation.

The function assign(values, s, d) will return the updated values (including the updates from constraint propagation), but if there is a contradiction--if the assignment cannot be made consistently--then assign returns False. For example, if a grid starts with the digits '77...' then when we try to assign the 7 to A2, assign would notice that 7 is not a possibility for A2, because it was eliminated by the peer, A1.

It turns out that the fundamental operation is not assigning a value, but rather eliminating one of the possible values for a square, which we implement with eliminate(values, s, d). Once we have eliminate, then assign(values, s, d) can be defined as "eliminate all the values from s except d".

In [4]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False


def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in PEERS[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in UNITS[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

### Test run

In [5]:
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = parse_grid(grid2)
display(sudoku_values)

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  



In [6]:
def grid_values(grid):

    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    assert len(grid) == 81, "Input grid must be a string of length 81 (9x9)"
    grid = [DIGITS if value == '.' else value for value in grid]
    return dict(zip(cross(ROWS, COLS), grid))
    pass


# Constraint Propagation - code refactoring

Try to decompose* eliminate function into smaller one.
* Decomposition is a process by which you can break down one complex function into multiple smaller functions. By doing this, you can solve for functions in shorter, easier-to-understand pieces.

In [7]:
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    global ASSIGNMENTS
    values[box] = value
    if len(value) == 1:
        ASSIGNMENTS.append(values.copy())
    return values

def eliminate(values):
    """
    Iterate through all squares and every time 
    if there is a square with one value, 
    then eliminate this value from the peers

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in PEERS[box]:
            values[peer] = values[peer].replace(digit,'')
    return values


def only_choice(values):
    """
    Iterate through all squares and every time
    if there is a square with a value that only fits in one square, 
    assign the value to this square

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for unit in UNITLIST:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                assign_value(values, dplaces[0], digit)

    return values

def reduce_puzzle(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    global ASSIGNMENTS
    ASSIGNMENTS = []
    solved = False
    while not solved:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        solved = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

display(reduce_puzzle(sudoku_values))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  



# Naked twins
http://www.sudokudragon.com/tutorialnakedtwins.htm


In [8]:
import itertools 


def naked_twins(values):
    """
    eliminate values using the naked twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers

    for unit in UNITLIST:
        unsolved = [box for box in unit if len(values[box]) > 1]
        pairs = list(itertools.combinations(unsolved, 2)) # indices of all pairs (0, 1), (0, 2), (0, 3), (0, 4),
        for i,j in pairs:
            chars1, chars2 = values[i], values[j] # the characters in each pair
            if len(chars1) ==  2 and chars1 == chars2: # if characters match, i.e. chars1 = '34' and chars2 = '34' they are twins
                not_twins = [box for box in unsolved if values[box] != chars1] # all boxes that are not the twins
                for box in not_twins:
                    for char in chars1: # remove the characters of the twins for each box that is not one of the twins
                        val = values[box].replace(char, '')
                        values = assign_value(values, box, val)

    return values


def solve_twins(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved = False
    while not solved:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        solved = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [39]:
def search(values, solver_type='naked_twins', attempts=[0]):
    if solver_type not in ['naked_twins', 'base']:
        raise TypeError('Not know solver type!')
    if solver_type == 'naked_twins':
        solver = solve_twins
    if solver_type == 'base':
        solver = reduce_puzzle
        
    values = solver(values)
    if values is False:
        return False 
    if all(len(values[s]) == 1 for s in SQUARES):
        return values 
    # Choose one of the unfilled squares with the fewest possibilities
    n, s = min([(len(values[s]), s) for s in SQUARES if len(values[s]) > 1])
    # Now use recurrence to solve each one of the resulting sudokus, and
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempts[0] = attempts[0] + 1
        attempt = search(new_sudoku, attempts=attempts, solver_type=solver_type)
        if attempt:
            return attempt

def solve(grid, solver_type='base', attempts=[0]):
    """
    Find the solution to a Sudoku grid.
    input:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
        solver_type(string): name of solver type 
        attempt(list): number of attempts took to resolve sudoku
    output: The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    global ASSIGNMENTS
    ASSIGNMENTS = []
    return search(grid_values(grid), solver_type, attempts=[0])


results = solve('2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3')
display(results)


2 3 9 |8 7 4 |1 5 6 
7 5 4 |3 1 6 |2 9 8 
6 8 1 |9 5 2 |3 7 4 
------+------+------
4 7 6 |1 2 8 |5 3 9 
3 1 2 |4 9 5 |6 8 7 
5 9 8 |6 3 7 |4 1 2 
------+------+------
1 4 3 |7 6 9 |8 2 5 
9 6 5 |2 8 3 |7 4 1 
8 2 7 |5 4 1 |9 6 3 



In [40]:
import time, random

def check_if_solved(sudoku):
    """
    Check if sudoku is solved
    input: A sudoku in dictionary form.
    output: boolean 
    """
    for x in sudoku:
        if (len(sudoku[x]) != 1):
            return False
    return True

def read_file(filename,spliter):
    """
    Read file by name and split lines
    input:
        filename(string): path to file 
        spliter(string): sign which split lines 
    """
    return open(filename).read().split(spliter)

def solve_all(filename, spliter, solver_type="base"):
    """
    Solve all puzzles in given file
    input:
        filename(string): path to file 
        spliter(string): sign which split lines
        solver_type(string): solver name
    """
    if solver_type not in ['naked_twins', 'base']:
        raise TypeError('Not know solver type!')

    time_list = []
    if_solved = []
    file = read_file(filename, spliter)
    for cnt, grid in enumerate(file):
        start_time = time.time()
        results = solve(grid, solver_type)
        time_list.append(time.time()-start_time)
        if_solved.append(check_if_solved(results))
    avg_time = sum(time_list)/len(time_list)
    print("Average time: ", avg_time, "s")
    print("Max time: ", max(time_list), "s")
    
    print("Solved of", filename, ": ", if_solved.count(True), "/", len(if_solved), "sudoku -", sum(if_solved)/len(if_solved) * 100, "%" )
    
    return avg_time

time_normal1 = []
time_twins1 = []
time_normal2 = []
time_twins2 = []
print("TWINS ALGORITHM: HARDEST \n")
solve_all("hardest.txt", "\n", "naked_twins")
print("\n")
print("TWINS ALGORITHM: TOP95 \n")
solve_all("top95.txt", "\n", "naked_twins")
print("\n")
print("NORMAL ALGORITHM: HARDEST \n")
solve_all("hardest.txt", "\n", "base")
print("\n")
print("NORMAL ALGORITHM: TOP95 \n")
solve_all("top95.txt", "\n", "base")

TWINS ALGORITHM: HARDEST 

Average time:  0.04417498906453451 s
Max time:  0.12163972854614258 s
Solved of hardest.txt :  12 / 12 sudoku - 100.0 %


TWINS ALGORITHM: TOP95 

Average time:  0.16575562577498587 s
Max time:  1.4091122150421143 s
Solved of top95.txt :  95 / 95 sudoku - 100.0 %


NORMAL ALGORITHM: HARDEST 

Average time:  0.045859197775522866 s
Max time:  0.09647226333618164 s
Solved of hardest.txt :  12 / 12 sudoku - 100.0 %


NORMAL ALGORITHM: TOP95 

Average time:  0.21837712589063143 s
Max time:  1.6032071113586426 s
Solved of top95.txt :  95 / 95 sudoku - 100.0 %


0.21837712589063143

In [47]:
def assign_org(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate_org(values, s, d2) for d2 in other_values):
        return values
    else:
        return False
    
def eliminate_org(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate_org(values, s2, d2) for s2 in PEERS[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in UNITS[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign_org(values, dplaces[0], d):
                return False
    return values
    
def random_puzzle(N=17):
    """Make a random puzzle with N or more assignments. Restart on contradictions.
    Note the resulting puzzle is not guaranteed to be solvable, but empirically
    about 99.8% of them are solvable. Some have multiple solutions."""
    values = dict((s, DIGITS) for s in SQUARES)
    for s in shuffled(SQUARES):
        if not assign_org(values, s, random.choice(values[s])):
            break
        ds = [values[s] for s in SQUARES if len(values[s]) == 1]
        if len(ds) == N:
            return ''.join(values[s] if len(values[s])==1 else '.' for s in SQUARES)
    return random_puzzle(N) ## Give up and make a new puzzle


def shuffled(seq):
    "Return a randomly shuffled copy of the input sequence."
    seq = list(seq)
    random.shuffle(seq)
    return seq


def make_sudoku(values,to_replace):
    gridvalues = []
    for x in values:
        xgrid = grid_values(x)
        for grid in xgrid:
            if xgrid[grid] == to_replace:
                xgrid[grid] = "123456789"
        gridvalues.append(xgrid)
    return gridvalues

In [50]:
print("\n\n RANDOMLY GENERATED \n\n")
random_grid = random_puzzle(N=17)
display(grid_values(random_grid))



 RANDOMLY GENERATED 


123456789 123456789 123456789 |123456789     9     123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     8     |123456789     2     123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
    2     123456789 123456789 |123456789     4     123456789 |    5     123456789 123456789 
123456789 123456789 123456789 |123456789     5     123456789 |    4     123456789 123456789 
123456789 123456789     4     |123456789     7     123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    6     123456789 123456789 |123456789     8     123456789 
123456789 123456789     9     |123456789     1     123456789 |    6     123456789 123456789 
123456789 123456789 123456789 |    4         

In [52]:
display(solve(random_grid, solver_type='base'))

7 1 8 |2 9 4 |3 5 6 
6 9 5 |1 3 8 |7 2 4 
4 2 3 |5 6 7 |8 1 9 
------+------+------
2 3 6 |8 4 1 |5 9 7 
9 7 1 |3 5 2 |4 6 8 
8 5 4 |9 7 6 |2 3 1 
------+------+------
1 4 7 |6 2 5 |9 8 3 
5 8 9 |7 1 3 |6 4 2 
3 6 2 |4 8 9 |1 7 5 

